# Example how to use Paituli STAC with Python

This example shows how to use [Paituli STAC catalogue](https://paituli.csc.fi/geoserver/ogc/stac/v1) with Python.  

This workflow is meant for processing big raster datasets, also with good support for time series. The main idea is to first define the search and processing as process graph. The downloading and processing is done lazily at the end, so that only needed data (good enough cloud-free image, only needed bands and area) is downloaded. The libraries take care of data download, so you do not need to know about file paths. These tools work best when data is provided as [Cloud-optimized GeoTiffs](https://www.cogeo.org/) (COGs).

Dask is used for parallelization of computing, see [CSC Dask tutorial](https://docs.csc.fi/support/tutorials/dask-python/), inc how to use Dask with Jupyter in Puhti web interface and how to create batch jobs with Dask.

The main steps:
* Start Dask cluster
* Query STAC catalogue to find images from area and time of interest and small cloud coverage, 
* Create first datacube, defining required bands and bbox.
* Mosaic the images with median value, for each month.
* Plot images
* Calculate monthly NDVI-index
* Save the NDVI-index to GeoTiff file.
* Close Dask cluster

At the end of the Notebook additionally is shown how to open files from STAC results with Rasterio.

The example is based on:
* [Stackstac documentation](https://stackstac.readthedocs.io/en/latest/basic.html),
* Stacspec.org, Tutorials, [Access Sentinel 2 Data from AWS plotting](https://stacspec.org/en/tutorials/access-sentinel-2-data-aws)

## Preparations

In [ ]:
import stackstac
import pystac_client
import pyproj
import xarray as xr
import numpy as np

### Define STAC endpoint

For using a specific STAC API, its endpoint must be defined.

Open the catalog description from the small black triangle, after running the cell.

In [ ]:
URL = "https://paituli.csc.fi/geoserver/ogc/stac/v1"
catalog = pystac_client.Client.open(URL)
catalog

See basic info about the STAC catalog, Which collections are available?

In [ ]:
collections_list = [(collection.title, collection.id) for collection in catalog.get_collections()]
collections_list.sort()
for collection in collections_list:
    print(collection[0] + ': ' + collection[1])

Use **collection ID** for search here and later. In this example we use:

* Sentinel-2 11-days data mosaics: **sentinel_2_11_days_mosaics_at_fmi** (original data as provided by ESA).

## Search


In [ ]:
%%time

lon, lat = 28.2, 63.62 #Tiilikkajärvi
buffer = 1

search = catalog.search(
    bbox=[lon,lat,lon+buffer,lat+buffer],
    collections=["sentinel_2_11_days_mosaics_at_fmi"],
    datetime="2021-08-01/2021-09-30"
)

item_collection = search.item_collection()
item_collection

## Retrieving data

### Get data to Xarray

We will use [Stackstac](https://stackstac.readthedocs.io/en) library for creating `Xarray DataArray` datacube from the STAC items. Alternatively, one could use [ODC STAC](https://odc-stac.readthedocs.io/en/latest/) or [Geowombad](https://geowombat.readthedocs.io/en/latest/index.html). There are some differences in details between the libraries, but in general they work in a similar way. See [ODC STAC discussion](https://github.com/opendatacube/odc-stac/issues/54), for differences between Stackstac and ODC STAC. 

Using the defaults, our data will be in its native coordinate reference system, at the finest resolution. But many also other values can be set here. 

* `bounds` - datacube bounds, use smaller bbox in data's UTM coordinate reference system, around the original search point and with width and height of `buffer`.
* `epsg` - datacube coordinate system, given as EPSG code.
* `chunksize` - how big part of data is analysed at once, see also [Dask chunksize](https://github.com/csc-training/geocomputing/blob/master/python/STAC/Readme.md#dask-chunksize). 
* `resolution` - pixel size
* See [stackstac.stac()](https://stackstac.readthedocs.io/en/latest/api/main/stackstac.stack.html#stackstac.stack) documentation for more details. 

This will be fast, because the actual data is not fetched yet. How does the datacube look like? How many dimensions does it have?

In [ ]:
%time 
x, y = pyproj.Proj("EPSG:3067")(lon, lat)
buffer = 2000 

cube = stackstac.stack(
    items=item_collection,
    bounds=(x-buffer, y-buffer, x+buffer, y+buffer), 
    assets=["b04", "b03", "b02", 'b08', "quality_scene_classification"],
    chunksize=(-1,1,2048,2048),
    resolution=10,
    # resampling=Resampling.bilinear
    epsg=3067
).squeeze() 
# When item_collection contains multiple epsg's, epsg value needs to be provided
cube

Remove pixels, which do not have data (clouds etc). For older mosaics this information is given in the asset `quality_scene_classification`. For later years, similar information is stored in asset `valid_observations`.

In [ ]:
sentinel_stack = xr.where((cube.sel(band="quality_scene_classification") >= 1), x = cube, y = np.nan)
sentinel_stack

Use xarray's `resample` to create 1-month median composites. Note that we still only work on metadata/lazy-loaded data, hence we have not downloaded any data yet.

In [ ]:
monthly = sentinel_stack.resample(time="MS").median("time", keep_attrs=True)
monthly

So far no data has been downloaded, nor anything computed with actual data. In this example the final data size is very small, but Dask is good also in handling much bigger amounts of data, also bigger than fits to memory.

It is also possible to [visualize, what Dask is going to do](https://docs.dask.org/en/stable/graphviz.html#). Sometimes some optimizations might be possible to make the graph better.

In [ ]:
import dask
dask.visualize(cube)

To start the data download and analysis process `compute()` could be used, but usually it is skipped and delayed even further until saving or plotting the data. The process can be followed from Dask Dashboard or Dask Lab Extension. Depending on the amount of data, this will take some time.

In [ ]:
# %%time
# data = monthly.compute()

Show the resulting images.

In [ ]:
%%time
monthly.sel(band=["b04", "b03", "b02"]).plot.imshow(row="time", rgb="band", robust=True, size=10);

In [ ]:
%%time
monthly.sel(band="quality_scene_classification").plot(row="time")